## Deliverable 2. Create a Customer Travel Destinations Map.

In [86]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [87]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Port Alfred,ZA,-33.59,26.89,48.99,73,0,10.67
1,1,Pimentel,PE,-6.84,-79.93,61.16,87,65,14.36
2,2,Hobart,AU,-42.88,147.33,48.20,49,40,14.99
3,3,Barrow,US,71.29,-156.79,39.20,86,90,16.11
4,4,Asfi,MA,32.30,-9.24,69.31,72,0,14.54


In [88]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [89]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Kidal,ML,18.44,1.41,84.07,59,28,4.68
10,10,Vanimo,PG,-2.67,141.30,85.87,62,14,4.34
12,12,Bals,RO,44.35,24.10,70.00,63,0,8.05
14,14,Kirakira,SB,-10.45,161.92,85.98,67,100,11.10
17,17,Atuona,PF,-9.80,-139.03,78.62,79,30,26.42
18,18,Kapaa,US,22.08,-159.32,86.00,55,1,17.22
19,19,Bambous Virieux,MU,-20.34,57.76,71.60,78,75,18.34
23,23,Mannarakkat,IN,10.98,76.47,77.00,88,40,1.14
32,32,Virginia Beach,US,36.85,-75.98,75.99,64,40,6.51
33,33,Faanui,PF,-16.48,-151.75,77.74,75,7,19.46


In [100]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [101]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [114]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Kidal,ML,84.07,18.44,1.41,
10,Vanimo,PG,85.87,-2.67,141.30,
12,Bals,RO,70.00,44.35,24.10,
14,Kirakira,SB,85.98,-10.45,161.92,
17,Atuona,PF,78.62,-9.80,-139.03,
18,Kapaa,US,86.00,22.08,-159.32,
19,Bambous Virieux,MU,71.60,-20.34,57.76,
23,Mannarakkat,IN,77.00,10.98,76.47,
32,Virginia Beach,US,75.99,36.85,-75.98,
33,Faanui,PF,77.74,-16.48,-151.75,


In [120]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")

In [122]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Kidal,ML,84.07,18.44,1.41,
10,Vanimo,PG,85.87,-2.67,141.30,Vahmoneh Lodge Vanimo
12,Bals,RO,70.00,44.35,24.10,Avangarde Hotel Restaurant
14,Kirakira,SB,85.98,-10.45,161.92,SanBiz Lodge
17,Atuona,PF,78.62,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...
672,Aquiraz,BR,78.80,-3.90,-38.39,
675,Damietta,EG,82.40,31.42,31.81,
676,Kuryk,KZ,71.98,43.20,51.65,
677,Tomatlan,MX,76.26,19.93,-105.25,


In [123]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)  
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df


,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Vanimo,PG,85.87,-2.67,141.30,Vahmoneh Lodge Vanimo
12,Bals,RO,70.00,44.35,24.10,Avangarde Hotel Restaurant
14,Kirakira,SB,85.98,-10.45,161.92,SanBiz Lodge
17,Atuona,PF,78.62,-9.80,-139.03,Villa Enata
18,Kapaa,US,86.00,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...
313,Vila Franca Do Campo,PT,73.40,37.72,-25.43,Pestana Bahia Praia
314,Lagoa,PT,73.98,39.05,-27.98,Casa Das Faias
318,Tavira,PT,73.40,37.13,-7.65,Hotel Porta Nova
319,Muli,MV,83.43,2.92,73.57,Medhufushi Island Resort


In [124]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search"


# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City ID")

In [125]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [126]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))